In [ ]:
""" CHANGES to v4
added Chiller Communication
"""

In [ ]:
import IBSEN
import csv
import numpy as np
import os
from datetime import datetime
import time
import Meerstetter
from ctypes import cdll,c_long, c_ulong, c_uint32,byref,create_string_buffer,c_bool,c_char_p,c_int,c_int16,c_double, sizeof, c_voidp
from TLPM import TLPM
#import serial

In [ ]:
def create_wavelengthaxis_and_dark_spectrum(exp_time):
    """ Creates a wavelengthaxis, a (dark) spectrum and reads the detectory type of a spectrometer
    
    Parameters: expTime : exposure time for (dark) spectrum (int)
    
    Returns: wavelengthaxis, dark_spectrum and detectortype
    """
    handle = IBSEN.InitSpectrometer(None, 0)
    dark_spectrum = IBSEN.acquireSpectra(handle, 0, exp_time, 1, -1, 1, -1, -1, 0, '', 0) 
    dark_spectrum = np.array(dark_spectrum).astype(np.float64)
    detector_length = IBSEN.get_numPixImage(handle)
    wavelengthaxis = IBSEN.createWavelengthAxis(handle, 0, detector_length-1)
    wavelengthaxis = np.array(wavelengthaxis).astype(np.float64)
    detector_type = IBSEN.get_detectorType(handle)
    IBSEN.closeDevice(handle)
    return wavelengthaxis, dark_spectrum, detector_type

In [ ]:
def create_handle(number, serialnumber):
    """ Creates a handle for a Spectrometer
    
    Parameters: serialnumber: serialnumber of the spectrometer (int)
                number: identifier for the spectrometer (int)
                
    Returns: handle: A handle to adress the Spectrometer
    
    """
    handle = IBSEN.InitSpectrometer(serialnumber, number)
    return handle    

In [ ]:
def save_data(filename, data_to_save):
    """Saves data in a csv file and checks if a file with the same name already exists
    
    Parameters: filename: filename where to save the data (str)
                data_to_save: list of lists with all the data to save (list)
                
    returns: filename: filename, since it is possible that the name got changed if the file already exists in the path for save
            
    """
    file_counter = 2
    while os.path.isfile(filename):
        filename = filename.replace('.csv', '')
        filename = filename +f"_v{file_counter}.csv"
        file_counter = file_counter + 1
        
    with open(filename, 'w', newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data_to_save)
    
    return filename

In [ ]:
def prepare_data_for_saving(pebble, spectra, burst_number, number_measurements, label_temp_TEC, label_power,label_temp_chiller):
    
    """ Prepares the recorded data from spectrometers, powermeter, Tec and Chiller for saving
    
    Parameters:
                pebble: pebble identifier to connect the right wavelengthaxis with the data
                spectra: all recorded spectras 
                burst_number: number of burst in each measurements 
                number_measurements: number of measurements during the run
                label_temp_TEC: list of the temperatures before each measurement
                label_power: list of the measured power before each measurement
                label_temp_chiller: list of measured chiller_temp 
                
    returns: a list of lists with the data ready to save with save_data function
    """
    
    data = [ [] for i in range(burst_number * number_measurements + 2)]
    counter_pixel = 0
    counter_burst_number = 0
    data[0] = list(range(0,256))
    data[0].extend(['Temp_TEC', 'Laserpower', 'T_Chiller'])
    #data[0].append('Laserpower')
    data[1] = spectrometer_data[pebble]['wavelengthaxis'] 

    for measurement in range(number_measurements):
        for burst in spectra[pebble][measurement]:
            for pixel in burst:
                if counter_pixel >= 256:
                    data[counter_burst_number + 2].extend([label_temp_TEC[measurement],label_power[measurement],
                                                           label_temp_chiller[measurement]])
                    #data[counter_burst_number + 2].append(label_power[measurement])
                    counter_pixel = 0
                    counter_burst_number = counter_burst_number + 1 
                data[counter_burst_number + 2].append(pixel)
                counter_pixel = counter_pixel + 1
                
    data[-1].extend([label_temp_TEC[-1], label_power[-1], label_temp_chiller[-1]])
    #data[-1].append(label_power[-1])
    return data

In [ ]:
def initialize_powermeter():
    """ Function to check for a connected powermeter.
    Opens a Connection if there is one. Sets the wavelengthrange to 1060 and power range to 22W.
    After this reads out the measured power
    
    Returns: power_label: Measured Power (c_double)
    """
    #calibrate Powermeter
    tlpm = TLPM()
    deviceCount = c_uint32()
    #get number of available powermeter
    tlpm.findRsrc(byref(deviceCount))
    print("devices found: " + str(deviceCount.value))
    #get resource name
    if deviceCount.value == 1:
        resourceName = create_string_buffer(1024)
        tlpm.getRsrcName(c_int(0), resourceName)
        print(c_char_p(resourceName.raw).value)
    else:
        print("No Powermeter or more than 1 detected")

    #open connection   
    tlpm.open(resourceName, c_bool(True), c_bool(True))
    #set wavelength to MEXl wavelength
    tlpm.setWavelength(c_double(1060))
    #set power range to 5W
    tlpm.setPowerRange(c_double(22.0))


    power_range = c_double()
    wavelength  = c_double()
    unit = c_int16()
    tlpm.getPowerRange(c_int(0), byref(power_range))
    tlpm.getPowerUnit(byref(unit))
    tlpm.getWavelength(c_int(0), byref(wavelength))


    if unit.value == 0:
        unit = "W"
    elif unit.value == 1:
        unit = "dbm"

    print(f"{power_range.value} {unit}, {wavelength.value} nm")
    power_label =  c_double()
    tlpm.measPower(byref(power_label))
    tlpm.close()
    
    return power_label

In [ ]:
def crc16_ModBus(data): 
    """CRC-16-MODBUS Hex Algorithm
    Parameters
    ----------
    data : string
        Data packets received.
    Returns
    -------
    str
        CRC as hex string
    
    Raises
    ----------
    ValueError
        If data packet in each index contains a byte > 256
    """
    data = bytearray.fromhex(data)
    crc = 0xFFFF
    
    # Calculate CRC-16 checksum for data packet
    for byte in data:
        crc ^= byte
        for _ in range(0, 8):
            lastbyte = crc & 0x0001
            crc = (crc >> 1) & 0x7fff
            if lastbyte:
                crc ^= 0xa001
                
    return ((crc >> 8) & 0xff), (crc & 0xff)

In [ ]:
def prepare_message_for_communication(message):
    """ prepares a message for communication with the chiller
    
    Parameters: message: a 6byte string 
    
    Returns: message: 8 byte bytearray ready to send to chiller.
                      The last two bytes are a checksum to ensure correct connection.
    """
    highbyte, lowbyte = crc16_ModBus(message)
    message = message + "0x{:02x}".format(lowbyte).split('x')[1] + "0x{:02x}".format(highbyte).split('x')[1]
    message = bytearray.fromhex(message)
    return message

In [ ]:
def read_and_write_chiller(text):
    import serial
    
    """ Write text to the Chiller and read the answer.
    
    Parameters: text: bytearry to send to the chiller
    
    Returns: reply: Reply from the Chiller as a bytearray
    """
    
    with serial.Serial('COM4', timeout=1) as ser:
        ser.write(text)
        reply = ser.readline()
        
    return reply 

In [ ]:
def enter_and_exit_program_mode(messages):
    
    """ Enter or exit the programme mode of the chiller
    
    parameters: messages: a list with the messages to send to enter or exit the programming mode
                            Messages to enter: ['010603000005', '010615000000']
                            Messages to exit: ['010603000006', '010616000000']
    
    Returns: 0: to indicate Success
    """
    
    while(True):
        
        message_prepared = prepare_message_for_communication(messages[0])
        reply = read_and_write_chiller(message_prepared)
        if message_prepared == reply:
            message_prepared_2 = prepare_message_for_communication(messages[1])
            reply_2 = read_and_write_chiller(message_prepared_2)
        if message_prepared_2 == reply_2:
            break
            
    return 0

In [ ]:
#Get number of connected spectrometer
numberOfSpectrometers = IBSEN.GetNumSpectrometers()
print("Es wurden {numberSpectrometers} Spektrometer detektiert.".format(numberSpectrometers = numberOfSpectrometers))

In [ ]:
if(numberOfSpectrometers > 0 ):
    #Determination of the available serial numbers
    pcb, serial = IBSEN.GetSerialNumbers(numberOfSpectrometers)
    #Print the serial numbers
    for i in range(numberOfSpectrometers):
        print("Spektrometer {i} hat PCB Nummer {pcb} und Seriennummer {serie}.".format(i= i, pcb=pcb[i], serie=serial[i]))

In [ ]:
#load parameter
params = {}
with open("parameter_files/params_burst.csv") as file:
    reader = csv.reader(file, delimiter = ',')
    for line in reader:
        params[line[0]] = line[1]
print(params)
        
#small relabelling for IBSEN
if int(params['timeScale']) == 1:
    params['timeScale'] = 'MILLI'
elif int(params['timeScale']) == 2:
    params['timeScale'] = 'MICRO'
print('Paramterfile loaded')
params

In [ ]:
##### Initialize a dict that has the serialnumbers as keys and a dict with the wavelengthaxis and dark_spectrum
#of the PEBBLE as values of the keys
# e.g. example_dict = {serialnumber : { 'wavelengthaxis' : [ 1, 2, 3, 4, 5, ,6], 'DarkSpec' : spectrum}}
# use this dict to add dark_spectrum and spectrum later on
#OBS! the function create_wavelengthaxis closes every handle that is given to the fucntion in order to work correct.
#You will need to open the handle again afterwards
spectrometer_data = {}

for i, serialnumber in enumerate(serial):
    wavelengthaxis, dark_spectrum, detector_type = create_wavelengthaxis_and_dark_spectrum(int(params['expTime']))
    spectrometer_data[i] = {'wavelengthaxis': wavelengthaxis, 'darkspec': dark_spectrum, 'detectortype': detector_type,
                            'serialnumber': serialnumber,}

#Get the detector type. Valid values are:
# 0: Hamamatsu S11639-01, S13496 or S14379 
# (ISB1)
# 1: Hamamatsu S11156 (ISB3)
# 2: Hamamatsu G11476 (ISB4)
# 3: Hamamatsu S10420 (ISB5)
# 4: Hamamatsu G13913 (ISB6
# 2 and 4 NIR 0, 1, 3 VIS

for item in spectrometer_data:
    if int(spectrometer_data[item]['detectortype']) == 2 or int(spectrometer_data[item]['detectortype']) == 4:
        spectrometer_data[item]['detectortype'] = 'NIR'
    else:
        spectrometer_data[item]['detectortype'] = 'VIS'
print("Created Wavelengthaxis, Type, DarkCounts and Handle for every connected Spectrometer ")

In [ ]:
#create Handles for every Spetrometer
for spectrometer in spectrometer_data:
    spectrometer_data[spectrometer].update(
        {'handle': create_handle(spectrometer, spectrometer_data[spectrometer]['serialnumber'] )})

In [ ]:
#set exposure Time of every spectrometer
if params['different_expTime_NIR_VIS'] == 'on':
    for spectrometer in spectrometer_data:
        if spectrometer_data[spectrometer]['detectortype'] == 'VIS':
            IBSEN.set_expTime(spectrometer_data[spectrometer]['handle'], int(params['expTimeVIS']),
                              timeUnit= params['timeScale'])
            print(f"Set exposure Time of Spectrometer {spectrometer} to {params['expTimeVIS']} {params['timeScale']}-seconds.")
        elif spectrometer_data[spectrometer]['detectortype'] == 'NIR':
            IBSEN.set_expTime(spectrometer_data[spectrometer]['handle'], int(params['expTimeNIR']),
                              timeUnit= params['timeScale'])
            print(f"Set exposure Time of Spectrometer {spectrometer} to {params['expTimeNIR']} {params['timeScale']}-seconds.")
    
elif params['different_expTime_NIR_VIS'] == 'off':
    for spectrometer in spectrometer_data:
        IBSEN.set_expTime(spectrometer_data[spectrometer]['handle'], int(params['expTime']),
                              timeUnit= params['timeScale'])
        print(f"Set exposure Time of Spectrometer {spectrometer} to {params['expTime']} {params['timeScale']}-seconds ")
    

In [ ]:
 #Before run make sure to correct subfolder in params file!!!
    
#set temperature to start value    
temperature = float(params["start_temp_TEC"])

temp_step =  float(params['temp_step'])
temp_start = 0
temp_end = 0

#check if temp_step 
if float(params["start_temp_TEC"]) > float(params["end_temp_TEC"]):
    if float(params['temp_step']) > 0:
        temp_step *= -1
        temp_start = float(params["start_temp_TEC"])
        temp_end = float(params["end_temp_TEC"]) - abs(temp_step)/2
    else:
        temp_start = float(params["start_temp_TEC"])
        temp_end = float(params["end_temp_TEC"]) - abs(temp_step)/2
        
          
elif float(params["start_temp_TEC"]) < float(params["end_temp_TEC"]):
    if float(params['temp_step']) < 0:
        temp_step *= -1
        temp_start = float(params["start_temp_TEC"])
        temp_end = float(params["end_temp_TEC"]) + abs(temp_step)/2
    else:
        temp_start = float(params["start_temp_TEC"])
        temp_end = float(params["end_temp_TEC"]) + abs(temp_step)/2
        
#number of temperature_steps
number_temperature_steps = int(abs((float(params["end_temp_TEC"])-float(params["start_temp_TEC"])))/abs(float(params['temp_step']))+1)

#counter for measurement protocol
counter_for_protocol = 0
#counter for header
counter_for_header = 0

#initialize measurement protcocol
time_protocol_measurement = [[] for i in range(int(params['numberMeasurements']) * int(number_temperature_steps) + 1)]
time_protocol_measurement[0].extend(["Messung", "Startzeit", "Endzeit", "Eingestellte Temperatur"])   

#make a list with every temperature specified by the range and the steps and round it to 1 decimal place
temperature_range = list(np.round(np.arange(temp_start, temp_end, temp_step), 1))
print(temperature_range)

In [ ]:
#calibrate TEC
# initialize controller
mc=Meerstetter.MeerstetterTEC(port ="COM3")
# get the values from DEFAULT_QUERIES
print(mc.get_data())
#set temperature to lowest temperature in run
mc.set_temp(float(params["start_temp_TEC"]))
print(mc.get_data()["target object temperature"])
#close connection
mc._tearDown()
#wait until the TEC has done its job
time.sleep(1)

#after 30 seconds check if temperature is okay
#check if temperature is correct
mc=Meerstetter.MeerstetterTEC(port ="COM3")
data = mc.get_data()
mc._tearDown()
temp, unit = data["object temperature"]
print(temp, unit)

In [ ]:
#check if connection to Chiller works
message_rdy_to_send = prepare_message_for_communication("0103001c0001")
reply = read_and_write_chiller(message_rdy_to_send)
temperature = (reply[3]*256 + reply[4])/10
print(temperature)

In [ ]:
###### loop over all temperatures
for temperature in temperature_range:
    
    print(f"set temperature to {temperature} degrees")  
    #set temperature for TEC
    #initialize controller
    mc=Meerstetter.MeerstetterTEC(port ="COM3")
    #set temperature
    mc.set_temp(float(temperature))
    #close connection
    print(f"Target object temperature: {mc.get_data()['target object temperature']}")
    mc._tearDown()
    
    #enter program mode
    enter_and_exit_program_mode(['010603000005', '010615000000'])

    #set temperature
    temperature_for_message = "0x{:04x}".format(int(temperature*10)).split('x')[1]
    set_temperature = f"0106007f{temperature_for_message}"
    set_temperature_prepared = prepare_message_for_communication(set_temperature)
    reply = read_and_write_chiller(set_temperature_prepared)
    print(set_temperature_prepared, reply)

    #exit program mode
    enter_and_exit_program_mode(['010603000006', '010616000000'])
    
    print(f"Wait 200 seconds for TEC and chiller to reach the target temperature: {temperature}")
    #wait 200s
    time.sleep(200)
    
    #declare lists for saving
    spectra = [[] for i in range(numberOfSpectrometers)]
    label_temp = [None for i in range(int(params['numberMeasurements']))]
    label_power = [None for i in range(int(params['numberMeasurements']))]
    label_t_chiller = [None for i in range(int(params['numberMeasurements']))]
    
    #loop over every measurement
    for i in range(int(params['numberMeasurements'])):
        
        #append measurement number to protocol
        time_protocol_measurement[counter_for_protocol + i + 1].append(i + 1)
        
        #connect to tec and read temperature for labelling
        mc=Meerstetter.MeerstetterTEC(port ="COM3")
        data = mc.get_data()
        label_temp[i], unit = data["object temperature"]
        mc._tearDown()
        
        #measure power from Thorlabs powermeter
        power_meas = initialize_powermeter()
        #save power label
        label_power[i] = power_meas.value
        
        #readout temperature from Chiller
        message_rdy_to_send = prepare_message_for_communication("0103001c0001")
        reply = read_and_write_chiller(message_rdy_to_send)
        temperature = (reply[3]*256 + reply[4])/10
        label_t_chiller[i] = temperature
        print(temperature)
        

        #get starting time_of_measurement for protocol
        start_time = time.ctime()
        time_protocol_measurement[counter_for_protocol + i + 1].append(start_time)
        print(f"Messung {i + 1} /", params['numberMeasurements'], f"bei Temperatur {label_temp[i]:.3f}")  
        print("Start : %s" % start_time)
        
        #make a burst with every connected pebble
        for j in range(numberOfSpectrometers):
            spectrum_burst = IBSEN.acquireSpectra_burst(spectrometer_data[j]['handle'],
                             numAcquisition = int(params['numberBurst']))
            spectra[j].append(spectrum_burst)
        
        #wait delay time
        time.sleep(int(params['delayTime']) * 10**(-3)) #because time.sleeps argument is in seconds, but in params file in ms
        
        # get end time of measurement for protocol
        end_time = time.ctime()
        time_protocol_measurement[counter_for_protocol + i + 1].append(end_time)
        print ("End : %s" % end_time)
        
        #add set temperature to protocol
        time_protocol_measurement[counter_for_protocol + i + 1].append(temperature)

    #save the captured spectra
    for pebble in spectrometer_data:
        filename = (f"{params['foldername']}" + '/' + f"{params['subfoldername']}" + '/' 
                    + f"{datetime.today().strftime('%Y-%m-%d-%H-%M')}_"
                    + f"PEBBLE{spectrometer_data[pebble]['serialnumber']}_SPECTRUMburst_T{temperature}.csv")
        #prepare_data_for_saving
        data = prepare_data_for_saving(pebble, spectra, int(params['numberBurst']), int(params['numberMeasurements']),
                                    label_temp, label_power, label_t_chiller)
        #save the spectra burst
        filename = save_data(filename, data)
        #update parameter file
        params.update({f'filename_{temperature}_{pebble}' : filename })
        #add Pebble with serialnumber to protocol only once
        if counter_for_header == 0:
            time_protocol_measurement[0].append(f"Pebble{spectrometer_data[pebble]['serialnumber']}")
            counter_for_header += 1
        #add filename to protocol
        time_protocol_measurement[counter_for_protocol + i + 1].append(filename)
    #increase protocol_counter by number of measurements
    counter_for_protocol = counter_for_protocol + int(params['numberMeasurements'])

In [ ]:
#before saving, add number_of_saved_spectra
number_of_saved_spectra = int(params['numberMeasurements']) * int(params['numberBurst'])
params.update({'number_of_saved_spectra' : number_of_saved_spectra})

#declare lists for parameter and values
parameter = []
value = []
#prepare parameter dict to right format for save_data function
for entry in params:
    parameter.append(entry)
    value.append(params[entry])
params_for_save = zip(parameter, value)

#save parameter file
filename_params = f"{params['foldername']}/{params['subfoldername']}/params_for_plot.csv"
save_data(filename_params,params_for_save)

#save protocol
filename_protocol = f"{params['foldername']}/{params['subfoldername']}/measurement_protocol.csv"
save_data(filename_protocol, time_protocol_measurement)    

In [ ]:
#close all handles
for spectrometer in range(len(spectrometer_data)):
    IBSEN.closeDevice(spectrometer_data[spectrometer]['handle'])
    print(spectrometer_data[spectrometer]['handle'])
print("All Spectrometer Closed!")